ETL PROJECT BY MELINDA EUDY, KELLY ROSE, DARRIUS SUMMERS MARCH 2021

In [222]:
#dependencies
import pandas as pd
import requests
import json
from pprint import pprint
from config import api_key, pgadmin_pass, pgadmin_user
from sqlalchemy import create_engine
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session


api_key= api_key
headers = {'Authorization': 'Bearer %s' % api_key}
url='https://api.yelp.com/v3/businesses/search''https://api.yelp.com/v3/businesses/search'

ImportError: cannot import name 'pgadmin_pass' from 'config' (C:\Users\kmhan\Documents\Bootcamp\Homework\ETL-Project\config.py)

STORE CSV INTO DATAFRAME ---KAGGLE - BREWERIES AND BEER PUBS IN THE US

In [201]:
csv_file = "Resources/7160_1.csv"
breweries_data_df = pd.read_csv(csv_file)
breweries_data_df.head()

,address,categories,city,country,key,lat,long,name,phones,postalCode,province,websites
0,407 Radam Ln,brewery,Austin,US,us/tx/austin/407radamln,NaN,NaN,(512) Brewing Co,5127072337,78745,TX,NaN
1,1135 N W Galveston Ave,Brewery and Bar,Bend,US,us/or/bend/1135nwgalvestonave,NaN,NaN,"10 Barrel Brewing Company, 10 Barrel Brewing Co","(541) 585-1007, 5415851007",97703,OR,10barrel.com
2,830 W Bannock St,"brewery, Restaurant, Pub, Brewery, pub, Brewer...",Boise,US,us/id/boise/830wbannockst,43.617711,-116.202883,"10 Barrel Brewing, 10 Barrel Brewing Co.","(208) 344-5870, 2083445870, 2.08344587E9",83702,ID,10barrel.com
3,1022 Texan Trl,"Wine Tours & Tastings, Food & Drink, Tours, Br...",Grapevine,US,us/tx/grapevine/1022texantrl,32.938385,-97.064340,10 Gallon Tours,"18174031832, 817 403-1832",76051,TX,http://www.10gallontours.com/
4,Po Box 135,"American Restaurants, Bar & Grill Restaurants,...",Villard,US,us/mn/villard/pobox135,NaN,NaN,10 Mile Tavern,3205542939,56385,MN,http://www.10miletavern.com


In [202]:
# Remove locations with missing data
breweries_data_df.dropna(inplace=True)
breweries_data_df.head()

,address,categories,city,country,key,lat,long,name,phones,postalCode,province,websites
2,830 W Bannock St,"brewery, Restaurant, Pub, Brewery, pub, Brewer...",Boise,US,us/id/boise/830wbannockst,43.617711,-116.202883,"10 Barrel Brewing, 10 Barrel Brewing Co.","(208) 344-5870, 2083445870, 2.08344587E9",83702,ID,10barrel.com
3,1022 Texan Trl,"Wine Tours & Tastings, Food & Drink, Tours, Br...",Grapevine,US,us/tx/grapevine/1022texantrl,32.938385,-97.064340,10 Gallon Tours,"18174031832, 817 403-1832",76051,TX,http://www.10gallontours.com/
10,3090 Shirley Dr,"Bars & Clubs, Breweries, Nightlife, Food & Drink",Jackson,US,us/mi/jackson/3090shirleydr/289346730,42.286010,-84.431260,127 Brewing,5172581346,49201,MI,https://www.facebook.com/127brewing
11,41 Lower Newton St,Brewers,Stalbans,US,us/vt/stalbans/41lowernewtonst,44.810713,-73.083558,14th Star Brewing Co Llc,8023931459,5478,VT,http://api.citygridmedia.com/content/places/v2...
15,189 Main St,"Restaurant, Restaurants, Bar, Brewery, and Res...",E Aurora,US,us/ny/eaurora/189mainst,42.767591,-78.627819,189 Public House,"7196528189, 7166528189",14052,NY,http://www.oneeightynine.com


In [203]:
# Delete Columns not required for final production database
del breweries_data_df["key"]
del breweries_data_df["long"]
del breweries_data_df["lat"]
del breweries_data_df["categories"]

# Reorder the columns to create better logic
breweries_data_df = breweries_data_df[["name", "phones", "address", "city", "province", "postalCode", "websites"]]
breweries_data_df.head()

,name,phones,address,city,province,postalCode,websites
2,"10 Barrel Brewing, 10 Barrel Brewing Co.","(208) 344-5870, 2083445870, 2.08344587E9",830 W Bannock St,Boise,ID,83702,10barrel.com
3,10 Gallon Tours,"18174031832, 817 403-1832",1022 Texan Trl,Grapevine,TX,76051,http://www.10gallontours.com/
10,127 Brewing,5172581346,3090 Shirley Dr,Jackson,MI,49201,https://www.facebook.com/127brewing
11,14th Star Brewing Co Llc,8023931459,41 Lower Newton St,Stalbans,VT,5478,http://api.citygridmedia.com/content/places/v2...
15,189 Public House,"7196528189, 7166528189",189 Main St,E Aurora,NY,14052,http://www.oneeightynine.com


In [204]:
# Reduce dataframe to North and South Carolina only
options = ['NC', 'SC'] 
breweries_data_df = breweries_data_df[breweries_data_df['province'].isin(options)]
breweries_data_df.head()

,name,phones,address,city,province,postalCode,websites
166,Aiken Brewing Company,8035020707,140 Laurens St S W,Aiken,SC,29801,aikenbrewingcompany.com
255,Amos Howard's Restaurant,"8282610011, 8283238753",2828 Us Highway 70 S W,Hickory,NC,28602,http://api.citygridmedia.com/content/places/v2...
351,Asheville Pizza Brewing Company,8282554077,77 Coxe Ave,Asheville,NC,28801,http://ashevillebrewing.com
352,Asheville Pizza Company,"8282540077, 8282541281, 8282545339",675 Merrimon Ave,Asheville,NC,28804,http://www.ashevillepizza.com
457,Bake Bottle & Brew,9103257550,147 N,Swansboro,NC,28584,bakebottlebrew.com


In [205]:
# Fix column names to prepare for merge with Yelp data
breweries_data_df=breweries_data_df.rename(columns ={"province": "state", "postalCode": "zip_code", "address": "address1"})
breweries_data_df.head()

,name,phones,address1,city,state,zip_code,websites
166,Aiken Brewing Company,8035020707,140 Laurens St S W,Aiken,SC,29801,aikenbrewingcompany.com
255,Amos Howard's Restaurant,"8282610011, 8283238753",2828 Us Highway 70 S W,Hickory,NC,28602,http://api.citygridmedia.com/content/places/v2...
351,Asheville Pizza Brewing Company,8282554077,77 Coxe Ave,Asheville,NC,28801,http://ashevillebrewing.com
352,Asheville Pizza Company,"8282540077, 8282541281, 8282545339",675 Merrimon Ave,Asheville,NC,28804,http://www.ashevillepizza.com
457,Bake Bottle & Brew,9103257550,147 N,Swansboro,NC,28584,bakebottlebrew.com


In [206]:
breweries_data_df.count()

name        77
phones      77
address1    77
city        77
state       77
zip_code    77
websites    77
dtype: int64

In [207]:
# Create dictornary search from YELP for bars in North Carolina
params = {'term':'bars','location':'NC'}

# Making a get request to the API
response=requests.get(url, params=params, headers=headers).json()


In [208]:
# Print response from YELP API for Bars in North Carolina
bars_nc = json.dumps(response, indent = 4, sort_keys=True)

response['businesses']

KeyError: 'businesses'

In [ ]:
bar_id = []
address1 = []
price = []
rating = []
review_count = []

for row in response['businesses']:
    try:
        bar_id.append(row['id']),
        address1.append(row['location']['address1']),
        price.append(row['price']),
        rating.append(row['rating']),
        review_count.append(row['review_count'])
        #print(f"{bar_id},{address1}, {price}, {rating}, {review_count}")
    except:
        pass

In [ ]:
# create dataframe for YELP bars in North Carolina
# Create DataFrame
data = {'address1':address1, 'bar_id': bar_id, 'price': price, 'rating': rating, 'review_count': review_count}
bars_nc_df = pd.DataFrame(dict([ (k,pd.Series(v)) for k,v in data.items() ]))
bars_nc_df.head()

In [ ]:
bars_nc_df.count()

In [ ]:
# Create dictornary search from YELP for bars in South Carolina
params = {'term':'bars','location':'SC'}

# Making a get request to the API
response=requests.get(url, params=params, headers=headers).json()


In [ ]:
# Print response from YELP API for Bars in South Carolina
bars_nc = json.dumps(response, indent = 4, sort_keys=True)

response['businesses']

In [ ]:
bar_id = []
address1 = []
price = []
rating = []
review_count = []

for row in response['businesses']:
    try:
        bar_id.append(row['id']),
        address1.append(row['location']['address1']),
        price.append(row['price']),
        rating.append(row['rating']),
        review_count.append(row['review_count'])
        #print(f"{bar_id},{address1}, {price}, {rating}, {review_count}")
    except:
        pass

In [209]:
# create dataframe for YELP bars in North Carolina
# Create DataFrame
data = {'address1':address1, 'bar_id': bar_id, 'price': price, 'rating': rating, 'review_count': review_count}
bars_sc_df = pd.DataFrame(dict([ (k,pd.Series(v)) for k,v in data.items() ]))
bars_sc_df.head()

<ipython-input-209-5d1242d6e484>:4: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  bars_sc_df = pd.DataFrame(dict([ (k,pd.Series(v)) for k,v in data.items() ]))


,address1,bar_id,price,rating,review_count


In [210]:
bars_sc_df.count()

address1        0
bar_id          0
price           0
rating          0
review_count    0
dtype: int64

In [211]:
rds_connection_string = f"{pgadmin_user}:{pgadmin_pass}@localhost:5432/Brews_db"
engine = create_engine(f'postgresql://{rds_connection_string}')

NameError: name 'pgadmin_user' is not defined

In [212]:
engine.table_names()

OperationalError: (psycopg2.OperationalError) FATAL:  password authentication failed for user "postgres"

(Background on this error at: http://sqlalche.me/e/13/e3q8)

In [213]:
breweries_data_df.to_sql(name='breweries_data', con=engine, if_exists='append', index=False)

OperationalError: (psycopg2.OperationalError) FATAL:  password authentication failed for user "postgres"

(Background on this error at: http://sqlalche.me/e/13/e3q8)

In [214]:
bars_nc_df.to_sql(name='bars_nc', con=engine, if_exists='append', index=False)

OperationalError: (psycopg2.OperationalError) FATAL:  password authentication failed for user "postgres"

(Background on this error at: http://sqlalche.me/e/13/e3q8)

In [215]:
bars_sc_df.to_sql(name='bars_sc', con=engine, if_exists='append', index=False)

OperationalError: (psycopg2.OperationalError) FATAL:  password authentication failed for user "postgres"

(Background on this error at: http://sqlalche.me/e/13/e3q8)

In [ ]:
pd.read_sql_query('select * from breweries_data', con=engine).head()

In [216]:
pd.read_sql_query('select * from bars_nc', con=engine).head()

OperationalError: (psycopg2.OperationalError) FATAL:  password authentication failed for user "postgres"

(Background on this error at: http://sqlalche.me/e/13/e3q8)

In [217]:
pd.read_sql_query('select * from bars_sc', con=engine).head()

OperationalError: (psycopg2.OperationalError) FATAL:  password authentication failed for user "postgres"

(Background on this error at: http://sqlalche.me/e/13/e3q8)